In [ ]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import numpy as np
import enum
import cv2
import time
import ipywidgets as widgets
import math 
from imutils import face_utils
from collections import OrderedDict
import argparse
import imutils
import dlib
from collections import deque
from ipywidgets import VBox, HBox, Label, Layout

import sys
sys.path.append('..')
from opencv_course.video import Camera

open_ratio = 0.145
left_dq = deque(maxlen=3)
right_dq = deque(maxlen=3) 

eye_html=widgets.HTML(
    value="睁眼"
)

face_html=widgets.HTML(
    value="前望"
)

In [ ]:
def bgr8_to_jpeg(value, quality=10):
    now = time.time()

    image = value
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        (x, y, w, h) = face_utils.rect_to_bb(rect)
        i=1

        left_dq.append(eye_ratio(shape[36], shape[39], shape[37], shape[38], shape[40], shape[41]))
        left_ratio = sum(left_dq) / float(len(left_dq))
        
        x1, y1 = shape[39]
        if left_ratio < open_ratio:
            cv2.putText(image, "Closed:{}".format(round(left_ratio, 2)), (x1 - 30, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
        else:
            cv2.putText(image, "Open:{}".format(round(left_ratio, 2)), (x1 - 30, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

        right_dq.append(eye_ratio(shape[42], shape[45], shape[43], shape[44], shape[46], shape[47]))
        right_ratio = sum(right_dq) / float(len(right_dq))
        x2, y2 = shape[45]
        if right_ratio < open_ratio:
            cv2.putText(image, "Closed:{}".format(round(right_ratio, 2)), (x2, y2 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
        else:
            cv2.putText(image, "Open:{}".format(round(right_ratio, 2)), (x2, y2 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
            
        for (x, y) in shape:
#             if i>=37 and i<=48:
            if right_ratio < open_ratio or left_ratio < open_ratio:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
            else:
                cv2.circle(image, (x, y), 1, (0, 255, 0), -1)
#             i+=1

        if left_ratio < open_ratio and right_ratio < open_ratio:
            eye_html.value="<div style='width=200px;height=100px;font-size:50px;padding:30px;background-color:red;color:white;text-align: center;'>闭眼</div>"
        else:
            eye_html.value="<div style='width=200px;height=100px;font-size:50px;padding:30px;background-color:green;color:white;text-align: center;'>睁眼</div>"
            
        face_degree = face_right(shape)
        if face_degree > 10:
            cv2.putText(image, "Right:{}".format(round(face_degree, 2)), (shape[27][0], shape[27][1]-40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
            face_html.value="<div style='width=100%;height=100px;font-size:50px;padding:30px;background-color:red;color:white;text-align: center;'>右望，角度:{}</div>".format(str(round(abs(face_degree), 2)))
        elif face_degree < -10:
            cv2.putText(image, "Left:{}".format(round(face_degree, 2)), (shape[27][0], shape[27][1]-40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
            face_html.value="<div style='width=100%;height=100px;font-size:50px;padding:30px;background-color:red;color:white;text-align: center;'>左望，角度:{}</div>".format(str(round(abs(face_degree), 2)))
        else:
            cv2.putText(image, "Normal:{}".format(round(face_degree, 2)), (shape[27][0], shape[27][1]-40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
            face_html.value="<div style='width=100%;height=100px;font-size:50px;padding:30px;background-color:green;color:white;'>前望，角度:{}</div>".format(str(round(abs(face_degree), 2)))
        
        
    cv2.putText(image, "processing time:{}s".format(round(time.time()-now, 2)), (10, 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

    return bytes(cv2.imencode('.jpg', image)[1])

In [ ]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def distance_p2line(p, l1, l2):
    return np.linalg.norm(np.cross(l2-l1, l1-p))/np.linalg.norm(l2-l1)

def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt(math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2) * 1.0) 

def eye_ratio(left, right, p1, p2, p3, p4):
    d1 = distance_p2line(p1, left, right)
    d2 = distance_p2line(p2, left, right)
    d3 = distance_p2line(p3, left, right)
    d4 = distance_p2line(p4, left, right)
    avg = (d1 + d2 + d3 + d4) / 4
    return avg / distance(left, right)

#判断是否右脸特别大
def face_right(shape):
    left_edge = shape[2]
    left_nose = shape[31]
    right_edge = shape[14]
    right_nose = shape[35]
    return (distance(right_edge, right_nose) - distance(left_edge, left_nose)) / (distance(right_edge, right_nose) + distance(left_edge, left_nose)) * 45 

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
shape_predictor = '../../detector/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

    
camera = Camera(width=320, height=240, fps=3)

image = widgets.Image(format='jpeg', width=800, height=600)  # this width and height doesn't necessarily have to match the camera


camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

label_layout = Layout(width='800px',height='700px')
display(HBox([image, VBox([eye_html, face_html])]))

In [ ]:
camera.unobserve_all()
camera.stop()